In [1]:
import pandas as pd
import numpy as np
import re

Loads CERT 6.2 email dataset

In [2]:
df = pd.read_csv("C:\\Users\\kaigo\\Downloads\\r6.2\\r6.2\\email.csv")

Loads answers for dataset and drops unecessary columns

In [3]:
answers_df = pd.read_csv("C:\\Users\\kaigo\\Downloads\\answers\\answers\\r6.2-3.csv", header=None)
answers_df = answers_df.where(answers_df[0] == 'email')
answers_df = answers_df[answers_df[0].notna()]
answers_df = answers_df.rename(columns={1: 'id', 2: "date", 3: "user", 4: "pc", 5: "to", 6: "cc", 7:"bcc", 8:"from", 9:"activity", 10:"size", 11: "attachment", 13: "content" })
answers_df = answers_df.drop([0, 12], axis=1)

Labels email data with malicious column

In [4]:
malicious_ids = answers_df['user'].unique()
df['malicious'] = [1 if x in malicious_ids else 0 for x in df['user']]

Feature Extraction

Gets number of words in message from message content

In [5]:
df['num_words'] = [len(x.split()) for x in df['content']]

Gets number of destinations and number of external destinations

In [6]:
# temp = []
# i = 0
# for _, x in df.iterrows():
#     total = 0
#     if type(x['to']) == str:
#         total += len(x['to'].split(";"))
#     if type(x['cc']) == str:
#         total += len(x['to'].split(";"))
#     if type(x['bcc']) == str:
#         total += len(x['to'].split(";"))
#     temp.append(total)
# df['num_dest'] = temp


In [7]:
df['num_dest'] = [len(re.findall("(@)", x['to'] + ";" + (x['cc'] if not pd.isnull(x['cc']) else '') + ";" + (x['bcc'] if not pd.isnull(x['bcc']) else ''))) for _, x in df[['to', 'cc', 'bcc']].iterrows()]

In [8]:
df['num_external'] = [len(set(re.findall("(?<=@)[^.]+(?=.)", x['to'] + ";" + (x['cc'] if not pd.isnull(x['cc']) else '') + ";" + (x['bcc'] if not pd.isnull(x['bcc']) else ''))) - set(re.findall("(?<=@)[^.]+(?=.)", x['to']))) for _, x in df[['to', 'cc', 'bcc']].iterrows()]

In [ ]:
df['num_external_bcc'] = [len(set(re.findall("(?<=@)[^.]+(?=.)", x['to'] + ";" + (x['bcc'] if not pd.isnull(x['bcc']) else ''))) - set(re.findall("(?<=@)[^.]+(?=.)", x['to']))) for _, x in df[['to', 'bcc']].iterrows()]

In [ ]:
df['num_attach'] = [len(x.split(";")) if type(x) == str else 0 for x in df['attachments']]

In [ ]:
df['attach_size'] = [sum(map(int, re.findall("\(([^()]+)\)", x))) if type(x) == str else 0 for x in df['attachments']]

In [ ]:
df['num_attach_type'] = [len(set(re.findall("\.(.*?)\(", x))) if type(x) == str else 0 for x in df['attachments']]

In [ ]:
df['num_bcc'] = [len(x.split(";")) if type(x) == str else 0 for x in df['bcc']]

In [ ]:
encoder_df = df[['num_dest','num_attach','num_attach_type', 'attach_size', 'num_external', 'num_external_bcc', 'size', 'num_words', 'num_bcc']]

In [ ]:
encoder_df

,num_dest,num_attach,num_attach_type,attach_size,num_external,num_external_bcc,size,num_words,num_bcc
0,3,0,0,0,0,0,45659,87,1
1,1,0,0,0,0,0,34142,81,0
2,2,2,1,1275148,1,0,1310925,108,0
3,1,0,0,0,0,0,23043,103,0
4,1,0,0,0,0,0,25210,168,0
...,...,...,...,...,...,...,...,...,...
10994952,1,1,1,537466,0,0,562755,110,0
10994953,2,0,0,0,1,0,27684,112,0
10994954,1,0,0,0,0,0,32292,53,0
10994955,4,0,0,0,0,0,23197,48,0


In [ ]:
y = df['malicious']
X = encoder_df

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
x = encoder_df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
scaled_df = pd.DataFrame(x_scaled)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.33, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Dropout, Embedding, LSTM
from tensorflow.keras.optimizers import RMSprop, Adam, Nadam
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import TensorBoard
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
encoding_dim=12
input_dim=X_train.shape[1]
inputArray=Input(shape=(input_dim,))
encoded=Dense(encoding_dim, activation='tanh')(inputArray)
decoded=Dense(input_dim, activation='softmax')(encoded)
autoencoder=Model(inputArray, decoded)
autoencoder.summary()